In [387]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
% matplotlib inline

import requests as re
from html.parser import HTMLParser

sns.set_context('poster')
sns.set_style('whitegrid')

In [388]:
df = pd.read_json('./rawData/train_data.json')

In [389]:
test = pd.read_json('./rawData/test_data.json')

In [390]:
test['test'] = True

In [391]:
df = df.append(test)
df.reset_index(drop=True, inplace=True)

In [392]:
df['test'].fillna(False, inplace=True)

In [393]:
df.columns = [c.replace(' ', '_') for c in df.columns]
df.columns = [c.replace('-', '_') for c in df.columns]
df.columns = [c.replace('/', '_') for c in df.columns]

In [394]:
df['bedrooms'] = df['bedrooms'].apply(float)
df['bedrooms'].fillna(0, inplace=True)
df['bathrooms'].fillna(0, inplace=True)

In [395]:
df.drop(['index', 'level_0'], axis=1, inplace=True)
df['interestVal'] = df['interest_level'].map({'high': 1, 'medium': 0.5, 'low':0})

In [396]:
class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

In [397]:
def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [398]:
def descrClean(x):
    des = strip_tags(x)
    return des.lower()

In [399]:
df['description'] = df['description'].apply(descrClean)

In [400]:
# Verify two laundry in buildings are the same
df['Laundry_In_Building'].value_counts()

0    46759
1     2593
Name: Laundry_In_Building, dtype: int64

In [401]:
# Verify two laundry in buildings are the same
df['Laundry_in_Building'].value_counts()

0    33008
1    16344
Name: Laundry_in_Building, dtype: int64

In [402]:
# Aggregate to create one laundry in building column that isn't case sensitive
df['laundry_in_building'] = df.apply(lambda row: row['Laundry_in_Building'] or row['Laundry_In_Building'], axis=1)

# Drop old laundry in building columns
df = df.drop(['Laundry_in_Building', 'Laundry_In_Building'], axis=1)

In [403]:
df.to_json('./cleaningTest/cleaned.json')

To determine the type of rental unit, we conduct a basic NLP

In [404]:
# Define basic unit types
apt = ['apartment', 'apt']
condo = ['condominium', 'condo']
walkUp = ['walk_up', 'walk-up', 'walkup', 'walk up']
studio = ['studio']
ph = ['ph', 'penhouse']
townhome = ['townhome', 'duplex', 'townhouse']
loft = ['loft']

types = [apt, condo, walkUp, studio, ph, townhome, loft]

In [405]:
# Function to Classify Unit Types
def unitType(x):
    homeType = {    
        }
    for lst in types:
        homeType[lst[0]] = False
    
    for lst in types:
        for w in lst:
            if w in x:
                return lst[0]
    return 'other'

In [406]:
df['type'] = df['description'].apply(lambda x : unitType(x)) #Determine rental type
df['foundType'] = ~df['type'].str.contains('other') #Determine if a type has been found

df = pd.concat([df, pd.get_dummies(df['type'])], axis=1) #Create binary columns for each type

#Combine and drop the two loft column
df['loft'].fillna(False, inplace=True)
df['loft'] = df[['loft', 'Loft']].apply(lambda row : row['loft'] or row['Loft'], axis=1)
df.drop('Loft', axis=1, inplace=True)

In [407]:
df.to_json('./cleaningTest/cleanedTyped.json')

# Generate interaction terms to find differentiators

In [408]:
# Luxury Score Term - higher the score means the more luxury items included
df['lux_score'] = (df['Exclusive'] + df['Doorman'] + df['Outdoor_Space'] + 
                   df['New_Construction'] + df['Roof_Deck'] + df['Fitness_Center'] + 
                   df['Swimming_Pool'] + df['Elevator'] + df['Laundry_in_Unit'] + df['Hardwood_Floors']) / 10

In [409]:
agentGroup = df.groupby(['manager_id']).mean()
buildingGroup = df.groupby(['building_id', 'manager_id']).mean()

In [410]:
buildingAvg = buildingGroup[['interestVal']]
buildingAvg.columns = ['prob_interest_building']
buildingAvg.reset_index(inplace=True)

In [411]:
managerAvg = agentGroup[['interestVal']]
managerAvg.columns = ['prob_interest_manager']
managerAvg.reset_index(inplace=True)

In [412]:
df = df.merge(managerAvg, on='manager_id', how='left')

In [413]:
df = df.merge(buildingAvg, on=['building_id', 'manager_id'], how='left')

In [414]:
def pBuildManager(pbuild, pmanager):
    if ~np.isnan(pbuild) and ~np.isnan(pmanager):
        return (pbuild + pmanager)/2
    return pmanager or pbuild

In [415]:
df['prob_buildManager'] = df.apply(lambda row: pBuildManager(row['prob_interest_building'], 
                                                             row['prob_interest_manager']), axis=1)

In [416]:
df['rooms'] = df['bedrooms']+df['bathrooms']
df['price_per_room'] = df['price']/df['rooms']

In [417]:
# Number of Luxury Features Term
df['num_luxury'] = df['Exclusive'] + df['Doorman'] + df['Outdoor_Space'] + df['New_Construction'] + df['Roof_Deck'] + df['Fitness_Center'] + df['Swimming_Pool'] + df['Elevator'] + df['Laundry_in_Unit'] + df['Hardwood_Floors']

In [418]:
# Number of Features per Listing
df['num_features'] = df['features'].apply(len)

In [419]:
# ADA compatible interaction term
# 1 if both elevator and wheelchair access, 0 if one or neither are included
df['ada'] = df['Elevator'] * df['Wheelchair_Access']

# Create transformed term that creates a score for outdoor spaces
# Higher the score, the more of these features are included
df['outdoor_score'] = (df['Outdoor_Space'] + df['Balcony'] + df['Common_Outdoor_Space'] 
                       + df['Garden_Patio'] + df['Roof_Deck'] + df['Terrace']) / 6

# Create interaction term for fitness oriented
# 1 if both swimming pool and fitness center are included, 0 if one or neither included
df['fitness_oriented'] = df['Fitness_Center'] * df['Swimming_Pool']

# Create interaction term for doorman/exclusive
# 1 if both are included, 0 if one or neither are included
df['door_excl'] = df['Doorman'] * df['Exclusive']

# Create interaction term for cats and dogs allowed
# 1 if both are allowed, 0 if one or neither are allowed
df['pets_allowed'] = df['Cats_Allowed'] * df['Dogs_Allowed']

In [420]:
df['price_per_feature'] = df['price']/df['num_features']
df['price_per_feature'].replace(np.inf, np.nan, inplace=True)

In [421]:
df['price_per_num_lux'] = df['price']/df['num_luxury']
df['price_per_num_lux'].replace(np.inf, np.nan, inplace=True)

In [422]:
g1 = df.groupby(['type']).mean()
g1.reset_index(inplace=True)

In [423]:
avgs = g1[['type','lux_score', 'num_features', 
           'num_luxury','outdoor_score', 'price_per_num_lux', 
           'price_per_feature']]

In [424]:
avgs.columns = ['avg_'+x for x in avgs]
avgs.rename(columns={'avg_type':'type'}, inplace=True)
df = pd.merge(df, avgs, on='type')

/Users/Braden/anaconda/lib/python3.6/site-packages/pandas/core/frame.py:2746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


In [425]:
df['price_per_num_lux'].fillna(df['avg_price_per_num_lux'], inplace=True)
df['outdoor_score'].fillna(df['avg_outdoor_score'], inplace=True)
df['lux_score'].fillna(df['avg_lux_score'], inplace=True)
df['price_per_feature'].fillna(df['avg_price_per_feature'], inplace=True)

df['price_lux_ratio'] = df['price_per_num_lux']/df['avg_price_per_num_lux']
df['outdoor_ratio'] = df['outdoor_score']/df['avg_outdoor_score']
df['lux_ratio'] = df['lux_score']/df['avg_lux_score']
df['price_feature_ratio'] = df['price_per_feature']/df['avg_price_per_feature']

In [426]:
df['numPhotos'] = df['photos'].apply(len)

In [427]:
train = pd.DataFrame(df[df['test']==False].dropna())
train.dropna(inplace=True)
train.drop('test', inplace=True, axis=1)
test = pd.DataFrame(df[df['test']])
test.reset_index(drop=True, inplace=True)
test.drop('test', inplace=True, axis=1)

In [428]:
train.to_json('./cleaningTest/train.json')
test.to_json('./cleaningTest/test.json')